<a href="https://colab.research.google.com/github/frantisco/IA-Agente-tributario/blob/main/Assessor_Tribut%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip -q install google-genai

In [7]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [9]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [13]:
##########################################
# --- Agente 1: Buscador Juridico --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um consultor jurídico.especializado em direito tributário.
        A sua tarefa é usar a ferramenta de busca do google (google_search) para manter seu conhecimento jurídico atuliazado,
        busque alterações no código tributário nacional, lei complementares e decretos.
        Busque notícias sobre a reforma tributária e como essa reforma impactará o sistema tributário brasileiro.
        Foque suas buscas nas mudanças legais mais impactantes dos ultimos 3 anos para quem está dentro do regime tributário do Simples Nacional.
        """,
        description="Agente que busca informações jurídicas no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [18]:
#########################################################################
# --- Agente 2: Consultor Juridico Municipal - Monte Santo de Minas --- #
#########################################################################
def agente_consultor(topico, lancamentos_buscados):
    consultor = Agent(
        name="agente_consultor",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Consultor #################################################
        instruction="""
        Você é um consultor jurídico especialista em direito tributário municipal na cidade de Monte Santo de Minas-MG.
        Com base nas informações de relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para manter seu conhecimento jurídico municipal atualizado,
        focando no código tributário municipal, código de postura e leis ordinárias.
        Com as informações necessárias coletadas você deve criar:
        Um plano para auxiliar o fiscal tributário municipal a monitorar empersas de Monte Santo de Minas no regime do simples nacional.
        Auxlie na fiscalização e criação de malhas finas usando como base no cruzamento de dados das notas fiscais eletronicas emitidas
        pelos contribuinte, com as informação coletadas pelo PGDAS e DASN SIMEI para os MEIs.
        Selecione os principais pontos de atenção para alertar os contribuintes, escritórios de contabilidade e contadores
        sobre as futuras ações fiscais que o fisco municipal dará início e faça um plano de post.

        """,
        description="Agente especializado em direito tributário municipal de Monte Santo de Minas",
        tools=[google_search]
    )

    entrada_do_agente_consultor = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(consultor, entrada_do_agente_consultor)
    return plano_do_post

In [19]:
############################################################################
# --- Agente 3: Redator do Post- Cmapanha Educativa para Autoregulação--- #
############################################################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a Prefeitura Municipal de MOnte Santo de Minas, sertor de arrecadação de tributos municipais.
            Utilize os pontos de ateção e alertas fornecidos no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para o Instagram sobre o tema indicado.
            O post deve ser linguagem clara e acessível, evitando jargões técnicos e expressões complexas, para garantir que a informação seja compreendida por todos
            e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts imformativos para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [20]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            O publico alvo são contribuintes na cidade de Monte Santo de Minas: empresários (ME e  EPP), MEIs (micro empreendedor invidual),
            escritórios de contabilidade e contadores.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [21]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts Informativo para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre temas solicitados {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_consultor(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Consultor) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts Informativo para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Limites de faturamento simples nacional
Maravilha! Vamos então criar o post sobre temas solicitados Limites de faturamento simples nacional

--- 📝 Resultado do Agente 1 (Buscador) ---



> Olá! Para te manter atualizado sobre os limites de faturamento do Simples Nacional e as mudanças mais recentes que podem impactar sua empresa nos últimos 3 anos, preparei um resumo com base nas informações mais recentes sobre a legislação e a reforma tributária.
> 
> 
> Com prazer, aqui está um resumo das informações mais relevantes sobre os limites de faturamento do Simples Nacional e as mudanças recentes que podem afetar sua empresa:
> 
> **Limites de Faturamento Atuais (2025)**
> 
> *   **Limite Anual:** O limite de faturamento anual para permanecer no Simples Nacional é de R$ 4,8 milhões.
> *   **MEI:** Para o Microempreendedor Individual (MEI), o limite de faturamento anual é de R$ 81 mil.
> *   **Sublimite Estadual:** Para o recolhimento de ICMS e ISS, alguns estados adotam um sublimite de R$ 3,6 milhões. Ultrapassar este sublimite pode exigir que o ICMS e o ISS sejam recolhidos fora do Simples Nacional, seguindo as regras do regime normal.
> 
> **Mudanças Propostas e Reforma Tributária**
> 
> *   **Propostas de Aumento:** Há discussões sobre o aumento do limite de faturamento anual para R$ 5 milhões. Existem projetos de lei em tramitação que propõem aumentar os limites para MEI (R$ 144.913,41), ME (R$ 869.480,43) e EPP (R$ 8.694.804,31).
> *   **Reforma Tributária e o Simples Nacional:** A reforma tributária de 2023 (Emenda Constitucional 132/2023), regulamentada pela Lei Complementar nº 214/2024, manteve o Simples Nacional, mas introduzirá mudanças relevantes.
> *   **Novos Impostos (IBS e CBS):** A reforma substituirá o ICMS, ISS, IPI, PIS e COFINS por dois novos impostos: o Imposto sobre Bens e Serviços (IBS) e a Contribuição sobre Bens e Serviços (CBS).
> *   **Simples Nacional Híbrido:** Empresas do Simples Nacional poderão optar por recolher o IBS e a CBS separadamente, no chamado "Simples Nacional Híbrido". Isso permitirá a apropriação de créditos de impostos, mas pode aumentar a carga tributária.
> *   **Conceito de Receita Bruta:** A definição de Receita Bruta foi ampliada para incluir "demais receitas da atividade ou objeto principal" da empresa.
> *   **Transição:** A transição para o novo sistema tributário ocorrerá gradualmente entre 2026 e 2033.
> 
> **Impactos e Como se Preparar**
> 
> *   **Efeitos Indiretos:** A implementação do novo sistema tributário trará efeitos indiretos que impactam a forma de apuração e a competitividade dos negócios.
> *   **Planejamento Tributário:** É recomendável que as empresas do Simples Nacional comecem a mapear seus processos, entender seus clientes e revisar suas estratégias fiscais para decidir se vale a pena continuar no mesmo regime tributário.
> *   **Regularização Fiscal:** A Receita Federal está intensificando a fiscalização e o processamento de dados. Empresas com pendências fiscais devem regularizar sua situação para evitar a exclusão do Simples Nacional.
> *   **Domicílio Tributário Eletrônico (DT-e):** O DT-e facilitará a comunicação entre a Receita Federal e as empresas, agilizando o processo de envio de notificações sobre irregularidades.
> 
> **Recomendações**
> 
> 1.  **Mantenha-se Informado:** Acompanhe as atualizações da legislação tributária e as regulamentações da reforma tributária para garantir que sua empresa esteja em conformidade.
> 2.  **Regularize Pendências:** Verifique e regularize quaisquer pendências fiscais para evitar a exclusão do Simples Nacional.
> 3.  **Planeje-se:** Elabore um planejamento tributário detalhado, considerando os impactos da reforma e as opções de tributação disponíveis.
> 4.  **Consulte um Especialista:** Busque o auxílio de um contador ou consultor tributário para avaliar a situação específica da sua empresa e tomar as melhores decisões.
> 
> Espero que estas informações sejam úteis para você!


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Consultor) ---



> Com certeza! Para auxiliar o fiscal tributário municipal de Monte Santo de Minas-MG a monitorar empresas no Simples Nacional, e com base nas informações sobre os limites de faturamento e mudanças recentes, preparei um plano detalhado.
> 
> 
> ### Plano Detalhado para Monitoramento de Empresas no Simples Nacional em Monte Santo de Minas-MG
> 
> Este plano visa auxiliar o fiscal tributário municipal a monitorar empresas optantes pelo Simples Nacional em Monte Santo de Minas, otimizando a fiscalização e garantindo a conformidade tributária.
> 
> #### 1. **Atualização Jurídica Municipal**
> 
> *   **Código Tributário Municipal:** Analisar o Código Tributário Municipal de Monte Santo de Minas (ex: Lei Municipal nº 927/89 e suas alterações) para identificar as competências e responsabilidades do município na fiscalização do Simples Nacional.
> *   **Código de Posturas:** Verificar o Código de Posturas (ex: Lei nº 18/1949) para identificar possíveis atividades sujeitas à fiscalização municipal e cruzamento de dados com as atividades declaradas no Simples Nacional.
> *   **Leis Ordinárias:** Acompanhar as leis ordinárias municipais que possam ter impacto na tributação e fiscalização de empresas do Simples Nacional.
> *   **Lei Orgânica Municipal:** Consultar a Lei Orgânica do Município para entender as competências tributárias do município e as normas sobre a administração tributária.
> 
> #### 2. **Cruzamento de Dados e Malha Fina**
> 
> *   **Notas Fiscais Eletrônicas (NF-e):**
>     *   **Monitoramento:** Coletar e analisar dados das NF-e emitidas pelas empresas do Simples Nacional no município.
>     *   **Cruzamento de Informações:** Comparar os valores declarados nas NF-e com as informações prestadas no Programa Gerador do Documento de Arrecadação do Simples Nacional (PGDAS) e na Declaração Anual Simplificada para o Microempreendedor Individual (DASN-SIMEI) para os MEIs.
>     *   **Identificação de Inconsistências:** Identificar divergências que possam indicar omissão de receitas ou outras irregularidades.
> *   **PGDAS e DASN-SIMEI:**
>     *   **Análise Detalhada:** Examinar as declarações do PGDAS e DASN-SIMEI para verificar a receita bruta declarada, os impostos recolhidos e as atividades econômicas informadas.
>     *   **Comparativo Setorial:** Comparar o desempenho das empresas com outras do mesmo setor para identificar possíveis desvios de padrão que justifiquem uma análise mais aprofundada.
> *   **Outros Dados:**
>     *   **Declarações Bancárias:** Em casos de suspeita de fraude, considerar a análise de movimentações bancárias para verificar a compatibilidade com a receita declarada.
>     *   **Informações de Cartões de Crédito/Débito:** Cruzar dados de operadoras de cartão com as declarações fiscais para identificar possíveis omissões de receita.
>     *   **Cadastros Municipais:** Verificar a consistência das informações cadastrais das empresas junto à prefeitura (endereço, atividade, etc.).
> 
> #### 3. **Pontos de Atenção para os Contribuintes**
> 
> 1.  **Limites de Faturamento:**
>     *   **MEI:** Faturamento anual de até R$ 81 mil.
>     *   **Simples Nacional:** Faturamento anual de até R$ 4,8 milhões.
>     *   **Sublimite Estadual:** Observar o sublimite de R$ 3,6 milhões para recolhimento de ICMS e ISS em alguns estados.
> 2.  **Definição de Receita Bruta:**
>     *   Estar atento à definição ampliada de Receita Bruta, que inclui "demais receitas da atividade ou objeto principal" da empresa.
> 3.  **Simples Nacional Híbrido:**
>     *   Avaliar a possibilidade de optar pelo recolhimento do IBS e CBS separadamente, considerando os impactos na carga tributária e a apropriação de créditos.
> 4.  **Regularização Fiscal:**
>     *   Manter a situação fiscal regularizada para evitar a exclusão do Simples Nacional.
>     *   Utilizar o Domicílio Tributário Eletrônico (DT-e) para facilitar a comunicação com a Receita Federal.
> 5.  **Planejamento Tributário:**
>     *   Realizar um planejamento tributário detalhado, considerando os impactos da reforma tributária e as opções de tributação disponíveis.
> 6.  **Acompanhamento da Legislação:**
>     *   Manter-se informado sobre as atualizações da legislação tributária e as regulamentações da reforma tributária.
> 
> #### 4. **Plano de Postagens (Redes Sociais e Site da Prefeitura)**
> 
> *   **Objetivo:** Informar e alertar os contribuintes sobre as ações de fiscalização e os pontos de atenção para conformidade tributária.
> *   **Conteúdo:**
>     1.  **Informativos:**
>         *   **Títulos:** "Atenção, Empresas do Simples Nacional de Monte Santo de Minas!", "Fique em Dia com o Fisco Municipal!"
>         *   **Conteúdo:** Explicação sobre os limites de faturamento, a importância da correta declaração de receitas e os riscos da sonegação fiscal.
>     2.  **Educacionais:**
>         *   **Títulos:** "Como Declarar Corretamente Suas Receitas no Simples Nacional", "Entenda o Cruzamento de Dados das NF-e com o PGDAS"
>         *   **Conteúdo:** Tutoriais e guias sobre como preencher corretamente as declarações, como emitir NF-e e como evitar erros comuns.
>     3.  **Alertas de Fiscalização:**
>         *   **Títulos:** "Fiscalização em Foco: Ações de Monitoramento do Simples Nacional em Monte Santo de Minas", "Malha Fina Municipal: Evite Surpresas!"
>         *   **Conteúdo:** Informar sobre as ações de fiscalização em andamento, os critérios utilizados para seleção de empresas e os procedimentos para regularização em caso de notificação.
>     4.  **Resultados da Fiscalização:**
>         *   **Títulos:** "Prefeitura de Monte Santo de Minas Intensifica a Fiscalização e Identifica Irregularidades no Simples Nacional", "Empresas Autuadas por Sonegação Fiscal: Veja o Que Aconteceu"
>         *   **Conteúdo:** Divulgar os resultados das ações de fiscalização, informando o número de empresas fiscalizadas, o valor dos impostos recuperados e as penalidades aplicadas.
>     5.  **Dicas e Boas Práticas:**
>         *   **Títulos:** "Dicas Para Manter Sua Empresa Regular no Simples Nacional", "Boas Práticas de Gestão Fiscal Para MEIs"
>         *   **Conteúdo:** Recomendações sobre como organizar a documentação fiscal, como realizar o planejamento tributário e como evitar problemas com o fisco.
> *   **Formato:**
>     *   Posts em redes sociais (Facebook, Instagram) com linguagem acessível e visualmente atraente.
>     *   Artigos no site da prefeitura com informações detalhadas e links para a legislação pertinente.
>     *   Vídeos explicativos com entrevistas de especialistas e representantes da prefeitura.
> *   **Frequência:**
>     *   Publicações semanais ou quinzenais, com temas variados e relevantes para os contribuintes.
> *   **Público-Alvo:**
>     *   Empresários optantes pelo Simples Nacional, MEIs, contadores, escritórios de contabilidade e demais interessados em questões tributárias municipais.
> 
> #### 5. **Implementação e Acompanhamento**
> 
> *   **Treinamento:** Capacitar os fiscais tributários para a utilização de ferramentas de análise de dados e para a interpretação da legislação tributária.
> *   **Software de Gestão:** Implementar ou otimizar o uso de softwares de gestão tributária que permitam o cruzamento de dados e a identificação de inconsistências de forma automatizada.
> *   **Comunicação:** Manter uma comunicação transparente e eficiente com os contribuintes, oferecendo canais de atendimento para esclarecimento de dúvidas e regularização de pendências.
> *   **Avaliação:** Monitorar os resultados das ações de fiscalização, avaliando o aumento da arrecadação, a redução da sonegação fiscal e o nível de satisfação dos contribuintes.
> *   **Comitê Gestor:** Criar um comitê gestor com representantes da prefeitura, dos contadores e dos empresários para discutir e propor melhorias no sistema tributário municipal.
> 
> Este plano detalhado, quando implementado de forma eficaz, permitirá ao fiscal tributário municipal otimizar a fiscalização das empresas do Simples Nacional em Monte Santo de Minas, garantindo a conformidade tributária, aumentando a arrecadação e promovendo um ambiente de negócios mais justo e transparente.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚨 Atenção, empreendedores de Monte Santo de Minas! 🚨
> 
> Fiquem ligados nos limites de faturamento do Simples Nacional para não terem surpresas! 🧐
> 
> 💰 MEI: limite de R$ 81 mil por ano.
> 🏢 Simples Nacional: até R$ 4,8 milhões anuais.
> 
> 👉 Ultrapassou? Regularize-se para evitar problemas com o fisco! 😉
> 
> Mantenha sua empresa em dia e aproveite os benefícios do Simples Nacional! 👍
> 
> #SimplesNacional #FiscalizaçãoMunicipal #MonteSantoDeMinas #Empreendedorismo


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos otimizar para torná-lo ainda mais atraente e informativo para o público-alvo. Aqui estão algumas sugestões:
> 
> **Problemas e Sugestões:**
> 
> *   **Clareza e Concisão:** O texto está bom, mas podemos adicionar um pouco mais de impacto e direcionamento.
> *   **Chamada para Ação:** Poderíamos ser mais específicos sobre o que os empreendedores devem fazer.
> *   **Hashtags:** Podemos adicionar hashtags mais direcionadas e relevantes.
> 
> **Rascunho Revisado Sugerido:**
> 
> 🚨 Alerta importante para MEIs e empresas em Monte Santo de Minas! 🚨
> 
> Você sabe quais são os limites de faturamento do Simples Nacional em 2024? 🤔 Não deixe que seu negócio seja pego de surpresa!
> 
> 💰 MEI: Faturamento anual de até R$ 81 mil.
> 🏢 Simples Nacional: Receita bruta anual até R$ 4,8 milhões.
> 
> ⚠️ Ultrapassou o limite? É crucial regularizar a situação para evitar multas e complicações com a Receita Federal.
> 
> Precisa de ajuda para entender ou regularizar sua situação fiscal? Entre em contato conosco e garanta a saúde financeira da sua empresa! 😉
> 
> 👉 [Link para contato/WhatsApp]
> 
> #SimplesNacional #MEI #Contabilidade #Fiscalização #MonteSantoDeMinas #EmpreendedorismoLocal #RegularizeSuaEmpresa
> 


--------------------------------------------------------------
